In [118]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd
import keyring
import numbers

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.pivot_table(df, values='a', index='b', columns='c', aggfunc='sum', fill_value=0)
# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [2]:
# Sign into ArcGIS Online
un = 'analytics_wfrc'
pw = keyring.get_password('Analytics AGOL', un)
gis = GIS(username=un, password=pw)

In [3]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [4]:
# def first_non_null_gen(lst):
#     return next((item for item in lst if item is not None), None)

In [5]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "wfrc_bike_map_features_TEST.gdb", 'results.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [38]:
roads = 'https://services1.arcgis.com/99lidPhWCzftIe9K/ArcGIS/rest/services/UtahRoads/FeatureServer/0'
trails = 'https://services1.arcgis.com/99lidPhWCzftIe9K/ArcGIS/rest/services/TrailsAndPathways/FeatureServer/0'
aoi = r'.\Inputs\WFRC_MPO_AOG_Counties.shp'
cities = r'.\Inputs\Cities.shp'

In [7]:
roads_lyr = arcpy.MakeFeatureLayer_management(roads, 'roads_lyr', where_clause=""" BIKE_PLN_L IS NOT NULL Or BIKE_PLN_R IS NOT NULL """)
trails_lyr = arcpy.MakeFeatureLayer_management(trails, 'trails_lyr', where_clause=""" DesignatedUses NOT IN ('Pedestrian') And 
                                                                                      Status NOT IN ('Future', 'Proposed', 'PROPOSED') And 
                                                                                      CartoCode NOT IN ('1 - Hiking Only', '7 - Steps') """)

arcpy.management.SelectLayerByLocation(roads_lyr, 'INTERSECT',  aoi)
arcpy.management.SelectLayerByLocation(trails_lyr, 'INTERSECT',  aoi)

<Result 'trails_lyr'>

In [9]:
# merge both roads and trails
bike_features = arcpy.management.Merge([roads_lyr, trails_lyr], output=os.path.join(gdb, 'bike_features'), add_source='ADD_SOURCE_INFO')

In [10]:
# add a unique id field
arcpy.management.AddField(bike_features, 'uid', "LONG")
arcpy.management.CalculateField(bike_features, "uid", '!OBJECTID!', "PYTHON3")

<Result '.\\Outputs\\wfrc_bike_map_features_TEST.gdb\\bike_features'>

In [39]:
# spatial join to walkshed areas
target_features = bike_features 
join_features = cities
output_features = os.path.join(gdb, "bf_cities_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

bf_cities_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, match_option="HAVE_THEIR_CENTER_IN")
bf_cities_sj_df = pd.DataFrame.spatial.from_featureclass(bf_cities_sj[0])[['uid','CITY']].copy()

In [11]:
# add directionality to line features
bf_dm = arcpy.stats.DirectionalMean(bike_features, os.path.join(gdb, "bike_features_directional_mean"), "DIRECTION", "uid")
#"CompassA"

In [12]:
bike_features_df = pd.DataFrame.spatial.from_featureclass(bike_features[0])
bf_dm_df = pd.DataFrame.spatial.from_featureclass(bf_dm[0])

In [33]:
bike_features_df.head(3)

,OBJECTID,STATUS,CARTOCODE,FULLNAME,FROMADDR_L,TOADDR_L,FROMADDR_R,TOADDR_R,PARITY_L,PARITY_R,PREDIR,NAME,POSTTYPE,POSTDIR,AN_NAME,AN_POSTDIR,A1_PREDIR,A1_NAME,A1_POSTTYPE,A1_POSTDIR,A2_PREDIR,A2_NAME,A2_POSTTYPE,A2_POSTDIR,QUADRANT_L,QUADRANT_R,STATE_L,STATE_R,COUNTY_L,COUNTY_R,ADDRSYS_L,ADDRSYS_R,POSTCOMM_L,POSTCOMM_R,ZIPCODE_L,ZIPCODE_R,INCMUNI_L,INCMUNI_R,UNINCCOM_L,UNINCCOM_R,NBRHDCOM_L,NBRHDCOM_R,ER_CAD_ZONES,ESN_L,ESN_R,MSAGCOMM_L,MSAGCOMM_R,ONEWAY,VERT_LEVEL,SPEED_LMT,ACCESSCODE,DOT_HWYNAM,DOT_RTNAME,DOT_RTPART,DOT_F_MILE,DOT_T_MILE,DOT_FCLASS,DOT_SRFTYP,DOT_CLASS,DOT_OWN_L,DOT_OWN_R,DOT_AADT,DOT_AADTYR,DOT_THRULANES,BIKE_L,BIKE_R,BIKE_PLN_L,BIKE_PLN_R,BIKE_REGPR,BIKE_NOTES,UNIQUE_ID,LOCAL_UID,UTAHRD_UID,SOURCE,UPDATED,EFFECTIVE,EXPIRE,CREATED,CREATOR,EDITOR,CUSTOMTAGS,GlobalID,TDMNET_L,TDMNET_R,PED_L,PED_R,PrimaryName,ID,DesignatedUses,SurfaceType,Class,OtherRestrictions,HorseAllowed,MotorizedAllowed,HikeDifficulty,BikeDifficulty,ADAAccessible,OwnerSteward,County,RecreationArea,SystemName,TransNetwork,Comments,DataSource,created_user,created_date,last_edited_user,last_edited_date,MERGE_SRC,uid,SHAPE
0,1,,11,SUNSET DR,2601.0,2609.0,2600.0,2608.0,None,None,E,SUNSET,DR,,2500,N,,,,,,,,,SW,SW,UT,UT,49011,49011,Clearfield,Clearfield,SYRACUSE,SYRACUSE,84075,84075,Syracuse,Syracuse,,,None,None,None,None,None,None,None,0,0,25.0,,,,,NaN,NaN,None,P,C,MU Syracuse,MU Syracuse,NaN,,None,,,,,None,,12TVL10104963_1425_S,,,,2024-02-14 21:42:24.000000,NaT,None,2015-09-02 23:08:13,TRANSADMIN,MICHAELFOULGER,,{EF7FAE26-6C7A-4B47-B9F9-8F4B7A1A3B0D},None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,roads_lyr,1,"{""paths"": [[[-12475547.9257, 5026071.866700001..."
1,2,,11,HELEN DR,8801.0,8819.0,8800.0,8818.0,None,None,W,HELEN,DR,,3185,S,,,,,,,,,SW,SW,UT,UT,49035,49035,Salt Lake City,Salt Lake City,MAGNA,MAGNA,84044,84044,,,MAGNA,MAGNA,None,None,None,None,None,None,None,0,0,25.0,,,,,NaN,NaN,None,P,B,MT Magna,MT Magna,NaN,,None,,,,,None,,12TVL06930627_HELEN_DR,12955,,,2021-09-15 15:16:01.000000,NaT,None,2015-09-02 23:08:13,TRANSADMIN,ENEEMANN,,{2CE612E5-DC01-4AB5-8EA4-1C9CD3A0881C},None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,roads_lyr,2,"{""paths"": [[[-12479065.0271, 4968506.503700003..."
2,3,,11,,0.0,0.0,0.0,0.0,None,None,,,,,,,,,,,,,,,SW,SW,UT,UT,49029,49029,Morgan,Morgan,MORGAN,MORGAN,84050,84050,,,,,None,None,None,None,None,None,None,0,0,25.0,,,,,NaN,NaN,None,,None,CO Morgan,CO Morgan,NaN,,None,,,,,None,,12TVL58382497,,,DLG,2016-03-29 22:17:20.000001,NaT,None,2015-09-02 23:08:13,TRANSADMIN,UDOT_EDIT,,{9E4DC9A2-BE79-4E4F-879E-95C387892818},None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,roads_lyr,3,"{""paths"": [[[-12412416.061900001, 4994446.2911..."


In [32]:
bf_dm_df.head(3)

,OBJECTID,CompassA,DirMean,CirVar,AveX,AveY,AveLen,TestStat,RefValue,PValue,UnifTest,uid,SHAPE,ExistingFacility1Side
0,1,269.639195,180.360805,1.110223e-16,-1.247562e+07,5.026071e+06,145.640334,1.5,5.991465,0.367879,Uniform,1,"{""paths"": [[[-12475547.9058, 5026071.493000001...",W
1,2,309.281801,140.718199,0.000000e+00,-1.248024e+07,5.110195e+06,759.782583,1.5,5.991465,0.367879,Uniform,10,"{""paths"": [[[-12479941.5781, 5109954.198200002...",NW
2,3,356.101680,93.898320,0.000000e+00,-1.247960e+07,5.058145e+06,279.522116,1.5,5.991465,0.367879,Uniform,100,"{""paths"": [[[-12479586.4404, 5058005.976899996...",N


In [123]:
def determine_direction(angle):
     
    if isinstance(angle, numbers.Number):
        
        if angle < 337.5 and angle > 292.5:
            direction = 'NW'
        elif angle == 337.5:
            direction = 'NNW'
        elif (angle >= 0 and angle < 22.5) or (angle > 337.5 and angle <= 360):
            direction = 'N'
        elif angle == 22.5:
            direction = 'NNE'
        elif angle < 67.5 and angle > 22.5:
            direction = 'NE'
        elif angle == 67.5:
            direction = 'ENE'
        elif angle < 112.5 and angle > 67.5:
            direction = 'E'
        elif angle == 112.5:
            direction = 'ESE'
        elif angle < 157.5 and angle > 112.5:
            direction = 'SE'
        elif angle == 157.5:
            direction = 'SSE'
        elif angle < 202.5 and angle > 157.5:
            direction = 'S'
        elif angle == 202.5:
            direction = 'SSW'
        elif angle < 247.5 and angle > 202.5:
            direction = 'SW'
        elif angle == 247.5:
            direction = 'WSW'
        elif angle < 292.5 and angle > 247.5:
            direction = 'W'
        elif angle == 292.5:
            direction = 'WNW'
    else:
        direction = 'NA'
    
    return direction    

In [45]:
bf_dm_df['Direction'] = bf_dm_df['CompassA'].map(determine_direction)
# bf_dm_df['Facility2_Side'] = bf_dm_df['Facility1Side'].map(determine_opposite_direction)

In [94]:
def determine_primary_bike_feature_and_side(_degrees, _bike_left=None, _bike_right=None, _planned_bike_left=None, _planned_bike_right=None):


    # lookup table for bike facility and rankings
    bike_lookup = {
    '1A':3, # 1A Cycle track, at-grade, protected with parking
    '1B':2, # 1B Cycle track, protected with barrier
    '1C':1, # 1C Cycle track, raised and curb separated (may be multiuse with peds)
    '1D':3, # 1D Cycle track, bi-directional
    '1E':3, # 1E Cycle track, center-running 
    '2A':4, # 2A Buffered bike lane
    '2B':5, # 2B Bike lane
    '2C':4, # 2C Bi-directional buffered bike lane
    '3A':6, # 3A Shoulder bikeway
    '3B':7, # 3B Marked shared roadway
    '3C':8, # 3C Signed shared roadway
    '1': 1, # 1 Cycle track, unspecified 
    '2':5, # 2 Bike lane, unspecified 
    '3':8, # 3 Other bike route, unspecified
    'PP':1, # Parallel Bike Path, Paved
    'PU':9, # Parallel Bike Path, Unpaved
    'UN':10, # Unknown Category
    }

    #-----------------------------------
    # Existing Bike Facilities
    #-----------------------------------

    # if there are bike features on both sides
    if _bike_left in bike_lookup.keys() and _bike_right in bike_lookup.keys(): 
        # get the rank for each facility
        bl_rank = bike_lookup[_bike_left]
        br_rank = bike_lookup[_bike_right]

        # if bike_left's facility is better than bike_right's or they tie
        if (bl_rank < br_rank) or (bl_rank == br_rank):
            primary_feature = _bike_left
            secondary_feature = _bike_right

            if _degrees >= 90:
                primary_feature_degrees = _degrees - 90
            if _degrees < 90:
                primary_feature_degrees = _degrees + 270

            if primary_feature_degrees <= 180:
                    secondary_feature_degrees = primary_feature_degrees + 180
            if primary_feature_degrees > 180:
                    secondary_feature_degrees = primary_feature_degrees - 180

        # if bike_right's facility is better than bike_left's
        if br_rank < bl_rank:
            primary_feature = _bike_right
            secondary_feature = _bike_left

            if _degrees <= 270:
                primary_feature_degrees = _degrees + 90
            if _degrees > 270:
                primary_feature_degrees = _degrees - 270

            if primary_feature_degrees <= 180:
                    secondary_feature_degrees = primary_feature_degrees + 180
            if primary_feature_degrees > 180:
                    secondary_feature_degrees = primary_feature_degrees - 180

    # if bike right does not have a facility
    elif _bike_left in bike_lookup.keys() and _bike_right not in bike_lookup.keys(): 

        primary_feature = _bike_left
        secondary_feature  = "NA"
        secondary_feature_degrees ='NA'

        if _degrees <= 270:
            primary_feature_degrees = _degrees + 90
        if _degrees > 270:
            primary_feature_degrees = _degrees - 270
    
    # if bike left does not have a facility
    elif _bike_right in bike_lookup.keys() and _bike_left not in bike_lookup.keys(): 
        
        primary_feature = _bike_right
        secondary_feature  = "NA"
        secondary_feature_degrees ='NA'

        if _degrees <= 270:
            primary_feature_degrees = _degrees + 90
        if _degrees > 270:
            primary_feature_degrees = _degrees - 270

    else:
         primary_feature = 'NA'
         secondary_feature = 'NA'
         primary_feature_degrees = 'NA'
         secondary_feature_degrees = 'NA'  


    #-----------------------------------
    # Planned Bike Facilities
    #-----------------------------------

    # are facilities planned for both sides?
    if _planned_bike_left in bike_lookup.keys() and _planned_bike_right in bike_lookup.keys(): 
        
        bl_rank = bike_lookup[_planned_bike_left]
        br_rank = bike_lookup[_planned_bike_right]

        # if bike_left's facility is better than bike_right's or they tie
        if (bl_rank < br_rank) or (bl_rank == br_rank):
            planned_primary_feature = _planned_bike_left
            planned_secondary_feature = _planned_bike_right

            if _degrees >= 90:
                planned_primary_feature_degrees = _degrees - 90
            if _degrees < 90:
                planned_primary_feature_degrees = _degrees + 270

            if planned_primary_feature_degrees <= 180:
                planned_secondary_feature_degrees = planned_primary_feature_degrees + 180
            if planned_primary_feature_degrees > 180:
                planned_secondary_feature_degrees = planned_primary_feature_degrees - 180

        # if bike_right's facility is better than bike_left's
        if br_rank < bl_rank:
            planned_primary_feature = _planned_bike_right
            planned_secondary_feature = _planned_bike_left

            if _degrees <= 270:
                planned_primary_feature_degrees = _degrees + 90
            if _degrees > 270:
                planned_primary_feature_degrees = _degrees - 270

            if planned_primary_feature_degrees <= 180:
                    planned_secondary_feature_degrees = planned_primary_feature_degrees + 180
            if planned_primary_feature_degrees > 180:
                    planned_secondary_feature_degrees = planned_primary_feature_degrees - 180

    # if a new facility is only planned for left
    elif _planned_bike_left in bike_lookup.keys() and _planned_bike_right not in bike_lookup.keys(): 
        
        bl_rank = bike_lookup[_planned_bike_left]
        
        # is there an existing facility on bike right?
        if _bike_right in bike_lookup.keys():

            br_rank = bike_lookup[_planned_bike_right]

            # if bike_left's facility is better than bike_right's or they tie
            if (bl_rank < br_rank) or (bl_rank == br_rank):
                planned_primary_feature = _planned_bike_left
                planned_secondary_feature = _bike_right

                if _degrees >= 90:
                    planned_primary_feature_degrees = _degrees - 90
                if _degrees < 90:
                    planned_primary_feature_degrees = _degrees + 270

                if planned_primary_feature_degrees <= 180:
                        planned_secondary_feature_degrees = planned_primary_feature_degrees + 180
                if planned_primary_feature_degrees > 180:
                        planned_secondary_feature_degrees = planned_primary_feature_degrees - 180

            # if bike_right's facility is better than bike_left's
            if br_rank < bl_rank:
                planned_primary_feature = _bike_right
                planned_secondary_feature = _planned_bike_left

                if _degrees <= 270:
                    planned_primary_feature_degrees = _degrees + 90
                if _degrees > 270:
                    planned_primary_feature_degrees = _degrees - 270

                if planned_primary_feature_degrees <= 180:
                        planned_secondary_feature_degrees = planned_primary_feature_degrees + 180
                if planned_primary_feature_degrees > 180:
                        planned_secondary_feature_degrees = planned_primary_feature_degrees - 180
        else:
            planned_primary_feature = _planned_bike_left
            if _degrees <= 270:
                    planned_primary_feature_degrees = _degrees + 90
            if _degrees > 270:
                    planned_primary_feature_degrees = _degrees - 270
          
    # if a new facility is only planned for right
    elif _planned_bike_right in bike_lookup.keys() and _planned_bike_left not in bike_lookup.keys(): 
        
        br_rank = bike_lookup[_planned_bike_right]
        
        # is there an existing facility on bike left?
        if _bike_left in bike_lookup.keys():

            bl_rank = bike_lookup[_planned_bike_left]

            # if bike_left's facility is better than bike_right's or they tie
            if (bl_rank < br_rank) or (bl_rank == br_rank):
                planned_primary_feature = _bike_left
                planned_secondary_feature = _planned_bike_right

                if _degrees >= 90:
                    planned_primary_feature_degrees = _degrees - 90
                if _degrees < 90:
                    planned_primary_feature_degrees = _degrees + 270

                if planned_primary_feature_degrees <= 180:
                        planned_secondary_feature_degrees = planned_primary_feature_degrees + 180
                if planned_primary_feature_degrees > 180:
                        planned_secondary_feature_degrees = planned_primary_feature_degrees - 180

            # if bike_right's facility is better than bike_left's
            if br_rank < bl_rank:
                planned_primary_feature = _planned_bike_right
                planned_secondary_feature = _bike_left

                if _degrees <= 270:
                    planned_primary_feature_degrees = _degrees + 90
                if _degrees > 270:
                    planned_primary_feature_degrees = _degrees - 270

                if planned_primary_feature_degrees <= 180:
                        planned_secondary_feature_degrees = planned_primary_feature_degrees + 180
                if planned_primary_feature_degrees > 180:
                        planned_secondary_feature_degrees = planned_primary_feature_degrees - 180
        else:
            planned_primary_feature = _planned_bike_right
            if _degrees <= 270:
                    planned_primary_feature_degrees = _degrees + 90
            if _degrees > 270:
                    planned_primary_feature_degrees = _degrees - 270
    else:
         planned_primary_feature = 'NA'
         planned_secondary_feature = 'NA'
         planned_primary_feature_degrees = 'NA'
         planned_secondary_feature_degrees = 'NA'
         

    return (primary_feature, secondary_feature, primary_feature_degrees, secondary_feature_degrees, planned_primary_feature, planned_secondary_feature, planned_primary_feature_degrees, planned_secondary_feature_degrees)



In [124]:
result = determine_primary_bike_feature_and_side(_degrees=0, _bike_left='1A', _bike_right='3C')
print(f"The primary bike feature is {result[0]} and is on the {determine_direction(result[2])} side")
print(f"The secondary bike feature is {result[1]} and is on the {determine_direction(result[3])} side")
print(f"The planned primary bike feature is {result[4]} and is on the {determine_direction(result[6])} side")
print(f"The planned secondary bike feature is {result[5]} and is on the {determine_direction(result[7])} side")

The primary bike feature is 1A and is on the W side
The secondary bike feature is 3C and is on the E side
The planned primary bike feature is NA and is on the NA side
The planned secondary bike feature is NA and is on the NA side


In [ ]:
bf_dm_df.loc[bf_dm_df['Direction'] == 'N', 'Facility1_Side'] = ''

In [46]:
bf_dm_df['Facility1_Side'].value_counts()

S     43412
W     38076
E     27880
N     23374
SE    11457
SW    10083
NW     7967
NE     7828
Name: Facility1_Side, dtype: int64

In [40]:
# list(bike_features_df.columns)

In [34]:
bike_features_df2 = bike_features_df[['FULLNAME','PrimaryName', 'CARTOCODE', 'BIKE_L',	'BIKE_R',	'BIKE_PLN_L',	'BIKE_PLN_R', 'CITY', 'County', 'SPEED_LMT', 'MERGE_SRC', 'SHAPE']].copy()

KeyError: "['CartoCode', 'CITY', 'SOURCE_ID'] not in index"

In [ ]:
roads2['NAME'] = np.nan
roads2['COUNTY'] = roads2['County']
roads2['CARTOCODE'] = roads2['CartoCode']

# roads2['ExistingFacility1'] = np.nan
# roads2['ExistingFacility1Side'] = np.random.choice(['N','S','E','W'], size=len(roads2))
# roads2['ExistingFacility2'] = np.nan
# roads2['ExistingFacility2Side'] np.random.choice(['N','S','E','W'], size=len(roads2))

# roads2['PlannedFacility1'] = np.nan
# roads2['PlannedFacilitySide1'] = np.random.choice(['N','S','E','W'], size=len(roads2))
# roads2['PlannedFacility2'] = np.nan
# roads2['PlannedFacilitySide2'] = np.random.choice(['N','S','E','W'], size=len(roads2))

roads2['STATUS'] = np.nan
roads2['SOURCE'] = np.nan
roads2['NOTES'] = np.nan

In [ ]:
roads2.loc[roads2['FULLNAME'].isna() == False, 'NAME'] = roads2['FULLNAME']
roads2.loc[roads2['PrimaryName'].isna() == False, 'NAME'] = roads2['PrimaryName']

roads2.loc[roads2['MERGE_SRC'].isin(['Bike_Trails_Pathways_Existing', 'Roads_Bike_Facilities_Existing']) == True, 'STATUS'] = 'EXISTING'
roads2.loc[roads2['MERGE_SRC'].isin(['Bike_Trails_Pathways_Planned', 'Roads_Bike_Facilities_Planned']) == True, 'STATUS'] = 'PLANNED'

roads2.loc[roads2['MERGE_SRC'].isin(['Roads_Bike_Facilities_Existing', 'Roads_Bike_Facilities_Planned']) == True, 'SOURCE'] = 'Utah_Roads'
roads2.loc[roads2['MERGE_SRC'].isin(['Bike_Trails_Pathways_Existing', 'Bike_Trails_Pathways_Planned']) == True, 'SOURCE'] = 'Trails_Pathways'

roads2.loc[roads2['STATUS'] == 'EXISTING', 'TYPE'] = first_non_null_gen([roads2['BIKE_L'], roads2['BIKE_R']])
roads2.loc[roads2['STATUS'] == 'PLANNED', 'TYPE'] = first_non_null_gen([roads2['BIKE_PLN_L'], roads2['BIKE_PLN_R']])

In [ ]:
roads3 = roads2[['NAME', 'CITY', 'COUNTY', 'CARTOCODE', 'ExistingFacility1', 'ExistingFacility1Side','ExistingFacility2', 'ExistingFacility2Side',,  'SIDE_1', 'SIDE_2', 'TYPE', 'STATUS', 'NOTES', 'SOURCE', 'SOURCE_ID', 'SHAPE']].copy()
roads3.spatial.to_featureclass(location=os.path.join(gdb, 'bike_features'), sanitize_columns=False)